In [1]:
from netCDF4 import Dataset
from datetime import datetime, date, timedelta 
import glob

In [2]:
nc = Dataset('./dataset.nc')
time = nc.variables['time'][:].astype(int)
U = nc.variables['ugos'][:]
V = nc.variables['vgos'][:]
lon = nc.variables['longitude'][:]
lat = nc.variables['latitude'][:]

In [3]:
ref = datetime(1950,1,1,0,0).toordinal() ## reference date 
time = time + ref
day_sat = [datetime.fromordinal(time[i]) for i in range(len(time))]

In [4]:
date = []
for i in range(len(day_sat)):
    date.append(str(day_sat[i])+'.000000 UTC') # converts into the required date format

In [5]:
prnt='/Users/baiyue/Desktop/Fsle_data/'   ## change this to where you want the daily data to be stored 

In [6]:
for i in range(len(date)): ## here we are creating one .nc file per day
    uu = U[i,:,:]
    vv = V[i,:,:]
    # create nc file
    dataset = Dataset(prnt+date[i][0:10].replace('-','_')+'.nc','w',format='NETCDF4_CLASSIC')
    # create the file dimension
    y=dataset.createDimension('y',lat.shape[0])
    x=dataset.createDimension('x',lon.shape[0])
    # create the variables
    y = dataset.createVariable('y',np.float32,('y',))
    x = dataset.createVariable('x',np.float32,('x',))
    u = dataset.createVariable('u',np.float32,('x','y'))
    v = dataset.createVariable('v',np.float32,('y','x'))
    # variable attributes
    y.long_name='Latitudes'
    y.units='degrees_north'
    x.long_name='Longitudes'
    x.units='degrees_east'

    u.long_name='U'
    u.units='cm/s'
    u.date= str(date[i])

    v.long_name='V'
    v.units='cm/s'
    v.date=str(date[i])
    ## write in the variables
    y[:]=lat
    x[:]=lon
    u[:]=100*uu.T # convert into cm/s  
    v[:]=100*vv  # convert into cm/s
    dataset.close()

PermissionError: [Errno 13] Permission denied: b'/Users/baiyue/Desktop/Fsle_data/2023_10_01.nc'

In [8]:
## Create the config.ini file
field = sorted(glob.glob(prnt +'2023*.nc'))
FF = list()
for i in range(len(field)):
    FF.append('U='+str(field[i]))
for i in range(len(field)):
    FF.append('V='+str(field[i]))

outF = open(prnt+str('config.ini'),'w') 
for line in FF:
  # write line to output file
  outF.write(line)
  outF.write("\n")
outF.write('U_NAME = u')
outF.write("\n")
outF.write('V_NAME = v')
outF.write("\n")
outF.write('FILL_VALUE = 0')
outF.close()